# EWMA

In [13]:
import pandas as pd
import numpy as np
from functools import partial
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

In [14]:
df = pd.read_excel('RiskMetrics.xlsx', skiprows = range(1))
no_of_stocks_owned = 1
current_asset_value_snp = 1
units_snp = 1/df['S&P BSE 500'].iloc[-1]
current_asset_value_nif = 1
units_nif = 1/df['Nifty'].iloc[-1]
df = (df.set_index('Date'))
df.head()


,S&P BSE 500,Nifty,Unnamed: 3,S&P Returns,Nifty Returns,Unnamed: 6,S&P Returns Sq,Nifty Returns Sq,Unnamed: 9,S&P (R),Nifty (R),Unnamed: 12,S&P (E),Nifty (E)
Date,,,,,,,,,,,,,,
2009-03-02,3139.09,2674.60,NaN,-0.018442,-0.019710,NaN,0.000340,0.000388,NaN,NaN,NaN,NaN,0.000000,0.000000
2009-03-03,3081.73,2622.40,NaN,0.002589,0.008657,NaN,0.000007,0.000075,NaN,NaN,NaN,NaN,0.000020,0.000023
2009-03-04,3089.72,2645.20,NaN,-0.025632,-0.026237,NaN,0.000657,0.000688,NaN,NaN,NaN,NaN,0.000020,0.000026
2009-03-05,3011.53,2576.70,NaN,0.008698,0.016722,NaN,0.000076,0.000280,NaN,NaN,NaN,NaN,0.000058,0.000066
2009-03-06,3037.84,2620.15,NaN,-0.018211,-0.018101,NaN,0.000332,0.000328,NaN,NaN,NaN,NaN,0.000059,0.000079


In [15]:
index = df.index
window = np.where(index> '2011-03-03')[0].min()
alpha = 0.06    # This is ewma's decay factor.
weights = list(reversed([(1-alpha)**n for n in range(window)]))

In [16]:
ewma = partial(np.average, weights=weights)

In [17]:
df['S&P (R)'] = df['S&P Returns Sq'].rolling(window).apply(ewma)
df['Nifty (R)'] = df['Nifty Returns Sq'].rolling(window).apply(ewma)

In [18]:
df.tail()

,S&P BSE 500,Nifty,Unnamed: 3,S&P Returns,Nifty Returns,Unnamed: 6,S&P Returns Sq,Nifty Returns Sq,Unnamed: 9,S&P (R),Nifty (R),Unnamed: 12,S&P (E),Nifty (E)
Date,,,,,,,,,,,,,,
2021-04-05,19580.089844,14637.799805,NaN,0.004720,0.003117,NaN,0.000022,0.000010,NaN,0.000151,0.000172,NaN,0.000159,0.000182
2021-04-06,19672.730469,14683.500000,NaN,0.009927,0.009189,NaN,0.000099,0.000084,NaN,0.000148,0.000167,NaN,0.000151,0.000172
2021-04-07,19868.990234,14819.049805,NaN,0.004849,0.003688,NaN,0.000024,0.000014,NaN,0.000140,0.000158,NaN,0.000148,0.000167
2021-04-08,19965.560547,14873.799805,NaN,-0.001077,-0.002622,NaN,0.000001,0.000007,NaN,0.000132,0.000149,NaN,0.000140,0.000158
2021-04-09,19944.060547,14834.849609,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
df.to_excel(r'Riskmetrics_after_forecasting.xlsx',index = False,  header=True)

In [20]:
z = stats.norm.ppf(0.99)

'''
S&P
'''
ewma_rolling_snp = round(np.sqrt(np.array(df['S&P (R)'])[-2]),4)
ewma_expanding_snp = round(np.sqrt(np.array(df['S&P (E)'])[-2]),4)
print("Standard Deviation of S&P BSE 500 using RiskMetrics Expanding Window : ",ewma_expanding_snp)
print("Standard Deviation of S&P BSE 500 using RiskMetrics Rolling Window : ",ewma_rolling_snp)
print("VaR of S&P BSE 500 using RiskMetrics Expanding Window : ",z*current_asset_value_snp*ewma_expanding_snp)
print("VaR of S&P BSE 500 using RiskMetrics Rolling Window : ",z*current_asset_value_snp*ewma_rolling_snp)

'''
NIFTY
'''
ewma_rolling_nif = round(np.sqrt(np.array(df['Nifty (R)'])[-2]),4)
ewma_expanding_nif = round(np.sqrt(np.array(df['Nifty (E)'])[-2]),4)
print("Standard Deviation of NIFTY using RiskMetrics Expanding Window : ",ewma_expanding_nif)
print("Standard Deviation of NIFTY using RiskMetrics Rolling Window : ",ewma_rolling_nif)
print("VaR of NIFTY using RiskMetrics Expanding Window : ",z*current_asset_value_nif*ewma_expanding_nif)
print("VaR of NIFTY using RiskMetrics Rolling Window : ",z*current_asset_value_nif*ewma_rolling_nif)

df = pd.DataFrame(columns=['VaR Terminology','S&P BSE 500','NIFTY'])
df = df.append({'VaR Terminology':'VaR with Expanding Window RiskMetrics','S&P BSE 500':z*current_asset_value_snp*ewma_expanding_snp,'NIFTY':z*current_asset_value_nif*ewma_expanding_nif}, ignore_index=True)
df = df.append({'VaR Terminology':'VaR with Rolling Window RiskMetrics(window ~ 5 years) ','S&P BSE 500':z*current_asset_value_snp*ewma_rolling_snp,'NIFTY':z*current_asset_value_nif*ewma_rolling_nif}, ignore_index=True)

Standard Deviation of S&P BSE 500 using RiskMetrics Expanding Window :  0.0118
Standard Deviation of S&P BSE 500 using RiskMetrics Rolling Window :  0.0115
VaR of S&P BSE 500 using RiskMetrics Expanding Window :  0.02745090491368192
VaR of S&P BSE 500 using RiskMetrics Rolling Window :  0.026753000551469668
Standard Deviation of NIFTY using RiskMetrics Expanding Window :  0.0126
Standard Deviation of NIFTY using RiskMetrics Rolling Window :  0.0122
VaR of NIFTY using RiskMetrics Expanding Window :  0.029311983212914595
VaR of NIFTY using RiskMetrics Rolling Window :  0.028381444063298258


In [21]:
df['Relative (Nifty/ S&P)'] = df['NIFTY']/df['S&P BSE 500']
df['Remarks'] = 0
for i in range(df.shape[0]): 
    if df['Relative (Nifty/ S&P)'][i]>=1:
        df['Remarks'][i] = "NIFTY is riskier"
    else:
        df['Remarks'][i] = "S&P is riskier"

In [23]:
df.to_csv (r'Results_RiskMetrics.csv',index = False,  header=True)

In [24]:
df

,VaR Terminology,S&P BSE 500,NIFTY,Relative (Nifty/ S&P),Remarks
0,VaR with Expanding Window RiskMetrics,0.027451,0.029312,1.067797,NIFTY is riskier
1,VaR with Rolling Window RiskMetrics(window ~ 5...,0.026753,0.028381,1.060870,NIFTY is riskier
